<a href="https://colab.research.google.com/github/gigiplz/Capstone/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import cv2
import numpy as np
import os
import csv
from tqdm import tqdm

# COCO 모델 기준 BODY_PARTS와 POSE_PAIRS
BODY_PARTS_COCO = {
    0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
    5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
    10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
    15: "LEye", 16: "REar", 17: "LEar"
}

POSE_PAIRS_COCO = [
    [1, 2], [1, 5], [2, 3], [3, 4], [5, 6], [6, 7],
    [1, 8], [8, 9], [9, 10], [1, 11], [11, 12], [12, 13],
    [1, 0], [0, 14], [14, 16], [0, 15], [15, 17]
]

# 모델 경로
protoFile = "/content/gdrive/My Drive/openpose_test/pose_deploy_linevec.prototxt"
weightsFile = "/content/gdrive/My Drive/openpose_test/pose_iter_440000.caffemodel"

# CSV 저장 경로 (영상 이름 기준)
csv_path = "/content/gdrive/My Drive/openpose_test/naruto_pose.csv"

# CSV 헤더 생성
if not os.path.exists(csv_path):
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Part', 'Confidence', 'X', 'Y'])

# 네트워크 불러오기
net_openpose = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

# Colab에서는 CUDA 사용 불가할 수 있으므로 주석 처리함
# net_openpose.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
# net_openpose.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

# 비디오 경로
video = "/content/gdrive/My Drive/openpose_test/naruto.mp4"
capture = cv2.VideoCapture(video)
total_frame = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))

points = []

def output_keypoints(threshold, BODY_PARTS):
    global points
    net = net_openpose

    image_height = 368
    image_width = 368

    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0),
                                       swapRB=False, crop=False)
    net.setInput(input_blob)
    out = net.forward()

    out_height = out.shape[2]
    out_width = out.shape[3]
    frame_height, frame_width = frame.shape[:2]
    points = []

    for i in range(len(BODY_PARTS)):
        prob_map = out[0, i, :, :]
        _, prob, _, point = cv2.minMaxLoc(prob_map)

        x = int((frame_width * point[0]) / out_width)
        y = int((frame_height * point[1]) / out_height)

        if prob > threshold:
            cv2.circle(frame_openpose, (x, y), 5, (0, 255, 255), thickness=-1)
            cv2.putText(frame_openpose, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1)
            points.append((x, y))

            with open(csv_path, 'a', newline='') as f:
                wr = csv.writer(f)
                wr.writerow([BODY_PARTS[i], prob, x, y])
        else:
            points.append(None)
            with open(csv_path, 'a', newline='') as f:
                wr = csv.writer(f)
                wr.writerow([BODY_PARTS[i], 0, 0, 0])

def output_keypoints_with_lines(POSE_PAIRS):
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]

        if points[partFrom] and points[partTo]:
            cv2.line(frame_openpose, points[partFrom], points[partTo], (0, 255, 0), 2)

# 프레임별 처리
for now_frame in tqdm(range(total_frame)):
    ret, frame = capture.read()
    if not ret:
        break

    frame_openpose = frame.copy()
    frame_zeros = np.zeros((frame.shape[0], frame.shape[1], 3), np.uint8)

    output_keypoints(threshold=0.1, BODY_PARTS=BODY_PARTS_COCO)
    output_keypoints_with_lines(POSE_PAIRS=POSE_PAIRS_COCO)

    # 이미지 저장
    cv2.imwrite(f"/content/gdrive/My Drive/openpose_test/test{now_frame:04d}.jpg", frame_openpose)

capture.release()

100%|█████████▉| 1037/1038 [1:34:02<00:05,  5.44s/it]
